In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np 
import spacy

nlp = spacy.load("en_core_web_sm")
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
# relative to this notebook
src_path = os.path.abspath(os.path.join('..'))
src = os.path.join(src_path,"src")
if src not in sys.path:
    sys.path.append(src)

In [ ]:
# Read data from json file
df = pd.read_csv('src_path/data/02_intermediate/Article20.csv')

# join heading and text column to single column
df.columns = ['date','heading','text','link','empty']
df['all']= df['heading'] + df['text']

df = df.drop(['date','heading','text','link','empty'], axis=1 )

df.head()

In [ ]:
df['processed'] = df['all'].apply(lambda x: nlp(x))

def is_token_allowed(token):
     '''
         Only allow valid tokens which are not stop words
         and punctuation symbols.
     '''
     if (not token or not token.string.strip() or
         token.is_stop or token.is_punct):
         return False
     return True

def preprocess_token(token):
     # Reduce token to its lowercase lemma form
     return token.lemma_.strip().lower()


def cleaning(row):
  #print(d)
  
  doc = df['processed'].values[0]
  #doc = row['processed']         #  <-------------------------------- WHY

  #for word in doc:#
  filtered_tokens = [preprocess_token(token)
       for token in doc if is_token_allowed(token)]
  return filtered_tokens


df['cleaned'] = df.applymap(lambda x: cleaning(x) )
#df['cleaned'] = df['processed'].apply(cleaning), also try to use map for efficiency

# Rejoining meaningful-stemmed to single snetence
def rejoin_words(row):
    my_list = row['cleaned']
    joined_words = ( " ".join(my_list))
    return joined_words

df['rejoined'] = df.apply(rejoin_words, axis=1)

In [ ]:
freq_words = []
# Iterate through rows
for index, row in df.iterrows():
    text = row['cleaned']
    word_freq = Counter(text)
    freq_words.append(word_freq.most_common(5))

print(freq_words[1])

In [ ]:
#article = []
#for w in mydoc:
  #if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
    #article.append(w.lemma_)

#print(article)


from nltk.stem import PorterStemmer
nltk.download('punkt')
stemming = PorterStemmer()


def identify_tokens(row):
    review = row['all']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['words'] = df.apply(identify_tokens, axis=1)


# WHOLE CLEAN UP
def stem_list(row):
    my_list = row['words']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

df['stemmed_words'] = df.apply(stem_list, axis=1)



# Removing stop words
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['stem_meaningful'] = df.apply(remove_stops, axis=1)


# Rejoining meaningful-stemmed to single snetence
def rejoin_words(row):
    my_list = row['stem_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

df['processed'] = df.apply(rejoin_words, axis=1)

df.head()

In [ ]:
#example_content = df.iloc[1]
#text = example_content['stem_meaningful']

#print(text)

#word_freq = Counter(text)
#common_words = word_freq.most_common(5)
#print(common_words)

for index, row in df.iterrows():
    print(row['stem_meaningful'])

list1 = []
# Iterate through rows
for index, row in df.iterrows():
    text = row['stem_meaningful']
    word_freq = Counter(text)
    list1.append(word_freq.most_common(5))

print(len(list1))
print(list1[0])
print(list1[0][0])
print(list1[0][0][0])


print(list1[1])

import matplotlib.pyplot as plt

word, freq = [], []
for item in list1:
  print(item)
  for itemm in item:
    word.append(itemm[0])
    freq.append(itemm[1])
  
plt.bar(word, freq)

In [ ]:
complete_text = ('Gus Proto is a Python developer currently'
     'working for a London-based Fintech company. He is'
    ' interested in learning Natural Language Processing.'
     ' There is a developer conference happening on 21 July'
     ' 2019 in London. It is titled "Applications of Natural'
     ' Language Processing". There is a helpline number '
     ' available at +1-1234567891. Gus is helping organize it.'
    ' He keeps organizing local Python meetups and several'
     ' internal talks at his workplace. Gus is also presenting'
     ' a talk. The talk will introduce the reader about "Use'
    ' cases of Natural Language Processing in Fintech".'
     ' Apart from his work, he is very passionate about music.'
    ' Gus is learning to play the Piano. He has enrolled '
     ' himself in the weekend batch of Great Piano Academy.'
    ' Great Piano Academy is situated in Mayfair or the City'
    ' of London and has world-class piano instructors.')

complete_doc = nlp(complete_text)

# Remove stop words and punctuation symbols
words = [token.text for token in complete_doc
if not token.is_stop and not token.is_punct] 

word_freq = Counter(words)
# 5 commonly occurring words with their frequencies
common_words = word_freq.most_common(5)
print (common_words)

# Unique words
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)